

#1. Installations et Importations




In [ ]:
!pip install -q faiss-cpu sentence-transformers transformers gradio langchain langgraph wikipedia langchain_community

import numpy as np
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import gradio as gr
from langchain.tools import tool
from langchain_community.utilities import WikipediaAPIWrapper
from langgraph.graph import StateGraph, START, END
from typing import Optional, TypedDict, Annotated
import operator
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
from google.colab import drive

# Monter Google Drive
drive.mount('/content/drive')


Mounted at /content/drive



#2.   Nettoyage des Données



In [ ]:
'''import pandas as pd
import ast
def load_and_clean(path):
    df = pd.read_csv(path)

    # Colonnes qui nous intéressent dans ce dataset
    columns = ["title", "overview", "genres", "vote_average", "release_date","poster_path"]
    existing_columns = [c for c in columns if c in df.columns]

    df = df[existing_columns]

    # Supprimer les lignes sans titre ou overview
    df = df.dropna(subset=["title","overview"])

    # Nettoyage simple
    df["overview"] = df["overview"].str.strip()
    if "genres" in df.columns:
        df["genres"] = df["genres"].fillna("[]")
    # Fill NaN values in poster_path with empty strings to prevent TypeError
    if "poster_path" in df.columns:
        df["poster_path"] = df["poster_path"].fillna("")

    return df
def clean_genres(g):
    try:
        l = ast.literal_eval(g)
        if isinstance(l, list):
            return " ".join([d.get("name", "") for d in l])
    except:
        pass
    return ""

if __name__ == "__main__":
    df = load_and_clean("/content/drive/MyDrive/movie-recommender-agent/data/raw/movies.csv")
    df["genres_clean"] = df["genres"].apply(clean_genres)
    df["vote_average"] = df["vote_average"].fillna(0)
    df["release_date"] = df["release_date"].astype(str)

    # Supprimer doublons
    df = df.drop_duplicates(subset=["title", "overview"])

    # Texte enrichi pour embeddings
    df["text"] = (
    df["title"] + " - " +
    df["genres_clean"] + " - " +
    "Note: " + df["vote_average"].astype(str) + " - " +
    "Année: " + df["release_date"].str[:4] + " - " +
    df["overview"]
)

    df.to_csv("/content/drive/MyDrive/movie-recommender-agent/data/processed/movies_clean.csv", index=False)
    df.head()

    print("Nettoyage amélioré OK")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-247020211.py", line 34, in <cell line: 0>
    df = load_and_clean("/content/drive/MyDrive/movie-recommender-agent/data/raw/movies.csv")
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-247020211.py", line 4, in load_and_clean
    df = pd.read_csv(path)
         ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/readers.py", line 1026, in read_csv
    return _read(filepath_or_buffer, kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/readers.py", line 626, in _read
    return parser.read(nrows)
           ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/readers.py", l

TypeError: object of type 'NoneType' has no len()




#3.  Géneration des Embeddings






In [ ]:
'''from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
import os

# Charger le dataset nettoyé
df = pd.read_csv("/content/drive/MyDrive/movie-recommender-agent/data/processed/movies_clean.csv")

# 🔥 Meilleur modèle multilingue pour FR + films
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Générer embeddings
embeddings = model.encode(
    df["text"].tolist(),
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True  # 🔥 normalisation intégrée !
)

# Sauvegarde
os.makedirs("/content/drive/MyDrive/movie-recommender-agent/data/embeddings", exist_ok=True)
np.save("/content/drive/MyDrive/movie-recommender-agent/data/multilingual_embeddings.npy", embeddings)

print("Embeddings améliorés sauvegardés !")
print("Shape :", embeddings.shape)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/32179 [00:00<?, ?it/s]

Embeddings améliorés sauvegardés !
Shape : (1029724, 384)





#4.  Indexation FAISS



In [ ]:
'''# =========================
# Construction Index FAISS
# =========================
!pip install -q faiss-cpu
import faiss
import numpy as np
import pandas as pd
import os

# Charger les embeddings sauvegardés
embeddings = np.load("/content/drive/MyDrive/movie-recommender-agent/data/multilingual_embeddings.npy")
df = pd.read_csv("/content/drive/MyDrive/movie-recommender-agent/data/processed/movies_clean.csv")

dim = embeddings.shape[1]  # taille des vecteurs

print("Dimension des embeddings :", dim)

# =========================
# 🔥 1) Index HNSW : plus rapide, meilleur rappel
# =========================
m = 32   # taille voisins
index = faiss.IndexHNSWFlat(dim, m)

# Normalisation obligatoire pour HNSW + cosine
faiss.normalize_L2(embeddings)

# Ajout des vecteurs
index.add(embeddings)
print(f"Index construit ! Nombre d'éléments : {index.ntotal}")

# =========================
# 🔥 2) Sauvegarde de l'index + metadata
# =========================
save_dir = "/content/drive/MyDrive/movie-recommender-agent/data/faiss_index"
os.makedirs(save_dir, exist_ok=True)

faiss.write_index(index, f"{save_dir}/index_hnsw.faiss")
df.to_csv(f"{save_dir}/metadata.csv", index=False)

print("Index FAISS + metadata sauvegardés !")

Dimension des embeddings : 384
Index construit ! Nombre d'éléments : 1029724
Index FAISS + metadata sauvegardés !


In [ ]:
# =========================
#  chemins
# =========================


BASE_PATH = '/content/drive/MyDrive/movie-recommender-agent'

RAW_CSV = f"{BASE_PATH}/data/raw/movies.csv"
PROCESSED_CSV = f"{BASE_PATH}/data/processed/movies_clean.csv"

# Embeddings déjà générés
EMB_NPY = f"{BASE_PATH}/data/multilingual_embeddings.npy"

# FAISS (nom réel de ton fichier)
FAISS_DIR = f"{BASE_PATH}/data/faiss_index"
FAISS_IDX = f"{FAISS_DIR}/index_hnsw.faiss"
METADATA_CSV = f"{FAISS_DIR}/metadata.csv"

os.makedirs(FAISS_DIR, exist_ok=True)


In [ ]:
# =========================
# Chargement du Dataset
# =========================
import pandas as pd

df = pd.read_csv(PROCESSED_CSV)
print("Loaded:", len(df), "rows")
df.head(2)


Loaded: 1029724 rows


,title,overview,genres,vote_average,release_date,poster_path,genres_clean,text
0,Inception,"Cobb, a skilled thief who commits corporate es...","Action, Science Fiction, Adventure",8.364,2010-07-15,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,NaN,Inception - - Note: 8.364 - Année: 2010 - Cob...
1,Interstellar,The adventures of a group of explorers who mak...,"Adventure, Drama, Science Fiction",8.417,2014-11-05,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,NaN,Interstellar - - Note: 8.417 - Année: 2014 - ...


In [ ]:
# =========================
#   Chargement des Embeddings
# =========================

from sentence_transformers import SentenceTransformer
import numpy as np

embeddings = np.load(EMB_NPY)
print("Embeddings loaded:", embeddings.shape)


Embeddings loaded: (1029724, 384)


In [ ]:
dim = embeddings.shape[1]

if os.path.exists(FAISS_IDX):
    index = faiss.read_index(FAISS_IDX)
else:
    faiss.normalize_L2(embeddings)
    index = faiss.IndexHNSWFlat(dim, 32)
    index.add(embeddings)
    faiss.write_index(index, FAISS_IDX)




#5.  Création des Tools Avec Langchain




In [ ]:
# =========================
#   Tools LangChain
# =========================
# On crée des "tools" utilisables ensuite par LangGraph; on garde le code simple.
from langchain.tools import tool
import numpy as np
from typing import Optional # Import Optional
wiki = WikipediaAPIWrapper(lang="fr")

# utilitaires
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
 # same model as embeddings

# 1) Search tool (FAISS)
@tool
def faiss_search(query: str, k: int = 10) -> dict:
    """Recherche les films les plus similaires à la requête donnée en utilisant l'index FAISS."""
    q_emb = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0].astype("float32")
    distances, indices = index.search(np.array([q_emb]), k)

    results = []
    for dist, idx in zip(distances[0], indices[0]):
        if idx < 0:
            continue
        row = df.iloc[int(idx)]
        results.append({
            "idx": int(idx),
            "title": row["title"],
            "genres": row.get("genres", ""),
            "overview": row.get("overview", ""),
            "vote_average": float(row.get("vote_average", 0)),
            "poster_path": row.get("poster_path", None),
            "distance": float(dist)
        })
    return {"results": results}


# 2) Metadata filter
@tool
def metadata_filter(items: list, genre: Optional[str] = None, min_year: Optional[int] = None, min_rating: Optional[float] = None) -> list:
    """Filtre une liste d'éléments (films) par genre, année minimale ou note minimale."""
    out = []
    for it in items:
        genres = (it.get("genres") or "").lower()
        year_ok = True
        rating_ok = True
        if genre:
            if genre.lower() not in genres:
                continue
        if min_year:
            try:
                year = int(str(it.get("release_date",""))[:4])
                if year < min_year: continue
            except:
                continue
        if min_rating:
            if it.get("vote_average", 0) < min_rating: continue
        out.append(it)
    return out

# 3) Reranker (par note pondérée par similarité)
@tool
def rerank(items: list, alpha_sim: float = 0.6) -> list:
    """Classe les éléments (films) en fonction d'un score combinant similarité et note moyenne."""
    # score final = alpha_sim * (1 / (1 + distance)) + (1-alpha_sim) * normalized_rating
    if not items: return []
    # normalize rating
    ratings = [it.get("vote_average", 0.0) for it in items]
    minr, maxr = min(ratings), max(ratings)
    def norm_rating(r):
        return (r - minr) / (maxr - minr + 1e-9)
    ranked = []
    for it in items:
        sim_score = 1.0 / (1.0 + it.get("distance", 1.0))
        rscore = norm_rating(it.get("vote_average", 0.0))
        final = alpha_sim * sim_score + (1-alpha_sim) * rscore
        it["_final_score"] = final
        ranked.append(it)
    ranked = sorted(ranked, key=lambda x: x["_final_score"], reverse=True)
    return ranked

# 4) Posters helper (compose full poster URLs if you use TMDB)
@tool
def posters(items: list, tmdb_base="https://image.tmdb.org/t/p/w300") -> list:
    """Ajoute les URLs complètes des affiches de film (posters) aux éléments."""
    out = []
    for it in items:
        poster = it.get("poster_path")

        try:
            poster = str(poster)
            if poster.lower() in ["nan", "none", "", "null"]:
                raise ValueError()
            it["poster_url"] = tmdb_base + poster
        except:
            it["poster_url"] = None

        out.append(it)
    return out

@tool
def wiki_search(query: str) -> str:
    """Recherche Wikipédia sans clé API"""
    try:
        return wiki.run(query)
    except:
        return "Aucun résultat Wikipedia."


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"

device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer_llm = AutoTokenizer.from_pretrained(MODEL_NAME)
model_llm = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)




tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
def llm_Explain(items: list, query: str, wiki_info: Optional[str] = None, max_new_tokens: int = 200) -> str:
    if not items:
        return "Désolé, aucun film trouvé."

    prompt = f"""
Tu es un assistant de recommandation de films.

Requête : "{query}"

"""
    if wiki_info:
        prompt += f"Informations contextuelles de Wikipedia sur la requête : {wiki_info}\n\n"

    prompt += f"""RÈGLE : ne renvoie que les sections suivantes, SANS rappeler le prompt :

### Analyse des films
### Pertinence
### Classement
### Recommandation finale

LISTE DES FILMS :
"""

    for it in items[:5]:
        prompt += f"- {it['title']} | genres: {it['genres']} | note: {it['vote_average']}\n"

    prompt += "\nRéponse :\n"

    inputs = tokenizer_llm(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)
    outputs = model_llm.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7
    )

    full = tokenizer_llm.decode(outputs[0], skip_special_tokens=True)

    # -----------------------------
    # 🔥 Extraire uniquement la partie APRÈS "Réponse :"
    # -----------------------------
    if "Réponse :" in full:
        full = full.split("Réponse :")[-1].strip()

    # supprimer toute trace du prompt
    for unwanted in ["Requête", "assistant", "LISTE DES FILMS", "Informations contextuelles de Wikipedia sur la requête"]: # Added wiki_info string to removal list
        if unwanted in full:
            full = full.split(unwanted)[0].strip()

    return full

In [ ]:
class MovieState(TypedDict):
    query: str
    raw_results: list
    ranked: list
    final_response: str

def node_search(state, config=None, runtime=None):
    q = state["query"]
    res = faiss_search.invoke({"query": q, "k": 15})
    return {"raw_results": res["results"]}

def node_rerank(state, config=None, runtime=None):
    ranked = rerank.invoke({"items": state["raw_results"]})
    return {"ranked": ranked}

def node_posters(state, config=None, runtime=None):
    return {"ranked": posters.invoke({"items": state["ranked"]})}

def node_wiki(state, config=None, runtime=None):
    q = state["query"]
    w = wiki_search.invoke({"query": q})
    return {"wiki": w}

def node_explain(state, config=None, runtime=None):
    resp = llm_Explain(state["ranked"], state["query"], wiki_info=state.get("wiki"))
    return {"final_response": resp}

builder = StateGraph(MovieState)
builder.add_node("search", node_search)
builder.add_node("rerank", node_rerank)
builder.add_node("posters", node_posters)
builder.add_node("wiki", node_wiki)
builder.add_node("explain", node_explain)

builder.add_edge(START, "search")
builder.add_edge("search", "rerank")
builder.add_edge("rerank", "posters")
builder.add_edge("posters", "wiki")
builder.add_edge("wiki", "explain")
builder.add_edge("explain", END)

agent = builder.compile()


In [ ]:
# =========================
# Invocation simple du graph (test)
# =========================
from langchain.messages import HumanMessage
import pandas as pd # Import pandas to use read_csv

# Ensure df is loaded. This is a safeguard if the preceding cell (LAAjJ-DKhUch) wasn't executed in the current session.
# PROCESSED_CSV should be defined from cell g2-t_brfhQ0m.
if 'df' not in globals():
    try:
        df = pd.read_csv(PROCESSED_CSV)
        print("DataFrame 'df' loaded successfully within Cell I.")
    except NameError:
        print("Error: PROCESSED_CSV variable is not defined. Please ensure setup cells are run.")
        # You might want to handle this more robustly, e.g., by raising an exception or exiting.
    except FileNotFoundError:
        print(f"Error: The file {PROCESSED_CSV} was not found. Please ensure data processing steps were completed.")

# Exemple simple : on place la query dans l'état initial puis invoque
state_in = {"query": "un film sur le voyage dans le temps", "raw_results": [], "filtered": [], "ranked": [], "final_response": ""}
result_state = agent.invoke(state_in)   # retourne dict d'état avec final_response
print("=== LLM explanation ===")
print(result_state.get("final_response"))
# Afficher top3 résultats
for r in result_state.get("ranked", [])[:3]:
    print("-", r["title"], "|", r.get("poster_url"), "| score:", r.get("_final_score"))

=== LLM explanation ===
Recommandation finale : The History of Time Travel, avec sa note de 6.5 qui souligne l'importance du voyage dans le temps dans la science-fiction et son intrigue engagée. 

Cependant, il est important de noter que le classement peut varier en fonction des critères utilisés pour la sélection des films (par exemple, certaines sources peuvent avoir une note différente ou plus élevée). Il est toujours préférable d'examiner davantage les détails des critiques si vous cherchez une information précise à ce sujet.
- The End of Time | https://image.tmdb.org/t/p/w300/8ERn9JiYSYolNKKZRTkSEH8B8M0.jpg | score: 0.8098262498503117
- The History of Time Travel | https://image.tmdb.org/t/p/w300/q1C0qZZ59j2Tsejd0SlFt5GPKIe.jpg | score: 0.768871054812878
- Tall Time Tales | https://image.tmdb.org/t/p/w300/nKmS6P5Dwbp3aeJZNj4Zg3vMzJL.jpg | score: 0.7536835727176139


In [ ]:
def chat_fn(query):
    state = agent.invoke({"query": query})
    ranked = state["ranked"][:12]
    explanation = state["final_response"]

    cards = []
    for r in ranked:
        poster = r.get('poster_url') or "https://via.placeholder.com/300x450?text=No+Image"
        cards.append(
            f"""
            <div style="
                width:180px;
                margin:10px;
                text-align:center;
                background:#1e1e1e;
                padding:10px;
                border-radius:12px;
                transition:0.3s;
            ">
                <img src="{poster}"
                     style="width:180px;height:260px;object-fit:cover;border-radius:10px;">
                <p style="color:white;font-weight:bold;font-size:14px;margin-top:8px;">
                    {r['title']}
                </p>
                <p style="color:#bbb;font-size:12px;margin:0;">
                    ⭐ {r.get('vote_average', 0)}
                </p>
            </div>
            """
        )

    grid = (
        "<div style='display:flex;flex-wrap:wrap;justify-content:center;"
        "background:#111;padding:15px;border-radius:12px;'>"
        + "".join(cards) +
        "</div>"
    )

    return explanation, grid



# =========================
# UI 🎬✨
# =========================
with gr.Blocks(
    theme=gr.themes.Soft(primary_hue="blue", secondary_hue="slate")
) as demo:

    gr.HTML("""
    <h1 style="text-align:center;margin-bottom:10px;font-size:40px;">
        🎬✨ <span style="color:#4A90E2;">Film Recommender Agent</span> ✨🎬
    </h1>
    <p style="text-align:center;font-size:18px;color:#444;">
        Trouvez le film parfait en un instant 🍿
        Tapez une requête .
    </p>
    """)

    with gr.Row():
        query = gr.Textbox(
            label="📝 Votre requête",
            placeholder="ex : film de voyage sur Mars...",
            lines=1,
            scale=4
        )
        btn = gr.Button("🔍 Rechercher", scale=1)

    with gr.Row():
        out_text = gr.Textbox(
            label="🧠 Analyse & Recommandation",
            lines=12,
            max_lines=20,
            show_copy_button=True,
            elem_id="big-explanation"
        )

    gr.Markdown("### 🎞️ Résultats similaires")
    gallery = gr.HTML()

    btn.click(chat_fn, inputs=[query], outputs=[out_text, gallery])

demo.launch()


/tmp/ipython-input-633609908.py:46: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2c43213e2bfc55bfdc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
